Classical cls-s on the winemag data were explored. 

(0) Setup

In [ ]:
!git clone https://github.com/kkonstantin182/wine-classifier.git
%cd wine-classifier
!pip install -r requirements.txt

Cloning into 'wine-classifier'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 149 (delta 83), reused 79 (delta 25), pack-reused 0
Receiving objects: 100% (149/149), 2.69 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/wine-classifier
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: scikit_learn
    Found existing installation:

In [ ]:
# Custom packages

import sys
from notebooks.path_conf import get_project_root
path_src = get_project_root() / "src"
sys.path.append(str(path_src.resolve()))

from src.text_processing import TextProcessing, Vectorization
from src.dataset import Dataset
from src.constants import SEED

In [ ]:
from google.colab import files

In [ ]:
# Libraries

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm
import multiprocessing as mp
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

# Other 
nltk.download('punkt') # Tokenization
!python -m spacy download it_core_news_sm # Lemmatization, stop words
!pip install catboost
import catboost as cb


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2023-05-14 08:49:08.245438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.4 MB/s eta 0:00:00


In [ ]:
from sklearn.compose import make_column_selector, make_column_transformer

(I) Data

In [ ]:
# Columns to use

TARGET_MAP = {
    # Removed for the privacy reasons
}

COLUMNS = {
    'target': ['character'],
    'text': ['text'],
    'numerical': ['alcohol', 'wine_name_length', 'n_grapes'],
    'categorical': ['category', 'region', 'is_complex_grape', 'alcohol_level']
}

In [ ]:
# Object for data cleaning
# As the data is cleaned already, only tokenizer will be used

tp_obj_clean = TextProcessing(is_lemmatized=True)

In [ ]:
# Loading data

train_set = pd.read_csv(r"/content/vh_train.csv", index_col=False)
test_set = pd.read_csv(r"/content/vh_test.csv", index_col=False)

print(train_set.shape)
print(test_set.shape)

(4016, 9)
(1005, 9)


In [ ]:
train_set.columns

Index(['target', 'text', 'alcohol', 'wine_name_length', 'n_grapes', 'category',
       'region', 'is_complex_grape', 'alcohol_level'],
      dtype='object')

In [ ]:
X_train, X_test = train_set.drop("target", axis=1), test_set.drop("target", axis=1)
y_train, y_test = train_set['target'], test_set['target']

# (II) Experiments


## (2.1) CatBoost with the custom feature processing 

In [ ]:
class AlcoholImputer(BaseEstimator, TransformerMixin):
    def __init__(self, group_column='category', impute_column1='alcohol', impute_column2='alcohol_level'):

        self.group_column = group_column
        self.impute_column1 = impute_column1
        self.impute_column2 = impute_column2
        self.transformed_columns_ = []

   
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = X.copy()
        X_transformed[self.impute_column1] = X.groupby(self.group_column)[self.impute_column1].apply(lambda x: x.fillna(x.median()))
        X_transformed[self.impute_column2] = X_transformed[self.impute_column1].apply(self.impute_alco_level)
        self.transformed_columns_ = X_transformed.columns.difference(X.columns).tolist()
        return pd.DataFrame(X_transformed, columns=X.columns, index=X.index)

    def impute_alco_level(self, x):
        if x < 11.5: level = 'low'
        elif 11.5 <= x < 13.5: level = 'medium'
        elif x >= 13.5: level = 'high'
        else: level = np.nan
        return level

    
    def get_params(self, deep=True):
        return {
            'group_column': self.group_column, 
            'impute_column1': self.impute_column1,
            'impute_column2': self.impute_column2}

    def set_params(self, **params):
        self.group_column = params['group_column']
        self.impute_column1 = params['impute_column1']
        self.impute_column2 = params['impute_column2']
        return self
    
    def get_feature_names_out(self, input_features=None):
        return self.transformed_columns_


In [ ]:
# Fill NaN in the alcohol column
alcohol_enc = AlcoholImputer().fit(X_train)
X_train = alcohol_enc.transform(X_train)
X_test = alcohol_enc.transform(X_test)

<ipython-input-11-10adc7537152>:15: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  X_transformed[self.impute_column1] = X.groupby(self.group_column)[self.impute_column1].apply(lambda x: x.fillna(x.median()))
<ipython-input-11-10adc7537152>:15: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  X_trans

In [ ]:
preprocessor1 = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(tokenizer=Vectorization.tokenize_it), "text"),
        ("num", MinMaxScaler(),  COLUMNS['numerical']),
        ("cat", OneHotEncoder(handle_unknown="ignore"), COLUMNS['categorical']),
        
    ]
)


In [ ]:
# Define pipelines for classifiers
pipelines = [
    ('CatBoost', Pipeline([
        ('preprocessor1', preprocessor1),
        ('clf', cb.CatBoostClassifier(random_state=SEED, iterations=30))
    ])),

]

# Define hyperparameters for grid search
hyperparameters = {
    'CatBoost': {
        'clf__iterations': [15],
        'clf__learning_rate': [0.001, 0.01, 0.1],
        'clf__depth': [6, 8, 10],
    },

}

In [ ]:
# Without hyp-s tuning
pipelines[0][1].fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Learning rate set to 0.084941
0:	learn: 2.4491687	total: 1.26s	remaining: 20m 57s
1:	learn: 2.2551079	total: 2.02s	remaining: 16m 46s
2:	learn: 2.0859961	total: 2.7s	remaining: 14m 56s
3:	learn: 1.9628824	total: 3.36s	remaining: 13m 56s
4:	learn: 1.8594195	total: 4.01s	remaining: 13m 18s
5:	learn: 1.7778022	total: 4.68s	remaining: 12m 55s
6:	learn: 1.7056888	total: 5.33s	remaining: 12m 36s
7:	learn: 1.6510827	total: 6.01s	remaining: 12m 25s
8:	learn: 1.5964572	total: 6.69s	remaining: 12m 16s
9:	learn: 1.5556103	total: 7.36s	remaining: 12m 8s
10:	learn: 1.5059772	total: 8.59s	remaining: 12m 52s
11:	learn: 1.4696155	total: 9.87s	remaining: 13m 32s
12:	learn: 1.4380549	total: 11.1s	remaining: 14m 4s
13:	learn: 1.4056546	total: 12.1s	remaining: 14m 12s
14:	learn: 1.3758861	total: 12.8s	remaining: 13m 58s
15:	learn: 1.3540411	total: 13.4s	remaining: 13m 46s
16:	learn: 1.3329313	total: 14.1s	remaining: 13m 35s
17:	learn: 1.3134419	total: 14.8s	remaining: 13m 26s
18:	learn: 1.2916715	total: 1

Pipeline(steps=[('preprocessor1',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(tokenizer=<function Vectorization.tokenize_it at 0x7f2b631abb50>),
                                                  'text'),
                                                 ('num', MinMaxScaler(),
                                                  ['alcohol',
                                                   'wine_name_length',
                                                   'n_grapes']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['category', 'region',
                                                   'is_complex_grape',
                                                   'alcohol_level'])])),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x7f2b40f88f10>)])

In [ ]:
# Make predictions on train data
y_train_pred_cb = pipelines[0][1].predict(X_train)
# Compute metrics on train data
train_accuracy_cb  = accuracy_score(y_train, y_train_pred_cb)
train_precision_cb , train_recall_cb , train_fscore_cb , train_support_cb  = precision_recall_fscore_support(y_train, y_train_pred_cb, average='macro')

print('CatBoost with the dafault val-s of hyp-s:')
print('Train Accuracy is: ', str(train_accuracy_cb))
print(
    'Train precision, recall, f1, support: ', 
    str(train_precision_cb), " ",
    str(train_recall_cb),  " ",
    str(train_fscore_cb), " ",
    str(train_support_cb))

CatBoost with the dafault val-s of hyp-s:
Train Accuracy is:  0.771414342629482
Train precision, recall, f1, support:  0.8323500610374965   0.7343840677892488   0.7633060009905845   None


In [ ]:
# Compute metrics on test data
y_test_pred_cb = pipelines[0][1].predict(X_test)
test_accuracy_cb = accuracy_score(y_test, y_test_pred_cb)
test_precision_cb, test_recall_cb, test_fscore_cb, test_support_cb = precision_recall_fscore_support(y_test, y_test_pred_cb, average='macro')

print('CatBoost with the dafault val-s of hyp-s:')
print('Test Accuracy is: ', str(test_accuracy_cb))
print(
    'Test precision, recall, f1, support: ', 
    str(test_precision_cb), " ",
    str(test_recall_cb),  " ",
    str(test_fscore_cb), " ",
    str(test_support_cb))

CatBoost with the dafault val-s of hyp-s:
Test Accuracy is:  0.6
Test precision, recall, f1, support:  0.6588271105728344   0.5656503126844462   0.584155049901732   None


Now with hyp-s tuning!

In [ ]:
# Define table to store results
results_table = pd.DataFrame(columns=[
    'Classifier', 
    'Hyperparameters', 
    'Train Accuracy', 
    'Train Precision', 
    'Train Recall', 
    'Train F1-score',
    'Test Accuracy', 
    'Test Precision', 
    'Test Recall', 
    'Test F1-score'])

# Train and evaluate models
for clf_name, pipeline in tqdm(pipelines, desc="Classifiers", total=len(pipelines)):
    print("Training", clf_name)
    clf = pipeline.named_steps['clf']
    hyperparams = hyperparameters[clf_name]
    rs = RandomizedSearchCV(pipeline, hyperparams, cv=5, scoring='accuracy', n_jobs=-1)
    # rs = GridSearchCV(pipeline, hyperparams, cv=5, scoring='accuracy', n_jobs=-1)
    rs.fit(X_train, y_train)
    
    # Make predictions on train data
    y_train_pred = rs.predict(X_train)
    
    # Compute metrics on train data
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision, train_recall, train_fscore, train_support = precision_recall_fscore_support(y_train, y_train_pred, average='macro')
    
    # Make predictions on test data
    y_test_pred = rs.predict(X_test)
    
    # Compute metrics on test data
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision, test_recall, test_fscore, test_support = precision_recall_fscore_support(y_test, y_test_pred, average='macro')

    # Store results in table
    results_table = results_table.append({
        'Classifier': clf_name,
        'Hyperparameters': rs.best_params_,
        'Train Accuracy': train_accuracy,
        'Train Precision': train_precision,
        'Train Recall': train_recall,
        'Train F1-score': train_fscore,
        'Test Accuracy': test_accuracy,
        'Test Precision': test_precision,
        'Test Recall': test_recall,
        'Test F1-score': test_fscore
    }, ignore_index=True)
    
    # Print progress bar
    remaining_iters = len(pipelines) - (pipelines.index((clf_name, pipeline)) + 1)
    print(f"{remaining_iters} iterations left")
    print("---------------------------------------------------------")

# Keep in mind that by default refit = True
# So, all metrics in the table are for the best found parameters 
  
# Print final results table
print("\nResults table:")
print(results_table)
results_table.to_csv('vh_cat_boost_hp_results.csv')
files.download('vh_cat_boost_hp_results.csv')


Classifiers:   0%|          | 0/1 [00:00<?, ?it/s]

Training CatBoost


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0:	learn: 2.4269875	total: 18s	remaining: 4m 12s
1:	learn: 2.2215075	total: 35.8s	remaining: 3m 52s
2:	learn: 2.0257787	total: 53.6s	remaining: 3m 34s
3:	learn: 1.8899733	total: 1m 13s	remaining: 3m 21s
4:	learn: 1.7922612	total: 1m 32s	remaining: 3m 4s
5:	learn: 1.7143292	total: 1m 50s	remaining: 2m 45s
6:	learn: 1.6296201	total: 2m 8s	remaining: 2m 26s
7:	learn: 1.5728365	total: 2m 27s	remaining: 2m 8s
8:	learn: 1.5075181	total: 2m 46s	remaining: 1m 50s
9:	learn: 1.4627730	total: 3m 3s	remaining: 1m 31s
10:	learn: 1.4188582	total: 3m 21s	remaining: 1m 13s
11:	learn: 1.3781525	total: 3m 39s	remaining: 54.9s
12:	learn: 1.3428950	total: 3m 58s	remaining: 36.7s
13:	learn: 1.3104799	total: 4m 17s	remaining: 18.4s
14:	learn: 1.2760313	total: 4m 35s	remaining: 0us


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-15-24b9f5e9e897>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
Classifiers: 100%|██████████| 1/1 [1:13:40<00:00, 4420.61s/it]


0 iterations left
---------------------------------------------------------

Results table:
  Classifier                                    Hyperparameters  \
0   CatBoost  {'clf__learning_rate': 0.1, 'clf__iterations':...   

   Train Accuracy  Train Precision  Train Recall  Train F1-score  \
0         0.58491         0.649976      0.509122        0.511107   

   Test Accuracy  Test Precision  Test Recall  Test F1-score  
0       0.518408        0.484362     0.455542       0.436607  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
results_table

,Classifier,Hyperparameters,Train Accuracy,Train Precision,Train Recall,Train F1-score,Test Accuracy,Test Precision,Test Recall,Test F1-score
0,CatBoost,"{'clf__learning_rate': 0.1, 'clf__iterations':...",0.58491,0.649976,0.509122,0.511107,0.518408,0.484362,0.455542,0.436607


In [ ]:
results_table['Hyperparameters'][0]

{'clf__learning_rate': 0.1, 'clf__iterations': 15, 'clf__depth': 10}

In [ ]:
hyperparameters = {
    'CatBoost': {
        'clf__iterations': [15],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__depth': [10],
    },

}

In [ ]:
# Define table to store results
results_table = pd.DataFrame(columns=[
    'Classifier', 
    'Hyperparameters', 
    'Train Accuracy', 
    'Train Precision', 
    'Train Recall', 
    'Train F1-score',
    'Test Accuracy', 
    'Test Precision', 
    'Test Recall', 
    'Test F1-score'])

# Train and evaluate models
for clf_name, pipeline in tqdm(pipelines, desc="Classifiers", total=len(pipelines)):
    print("Training", clf_name)
    clf = pipeline.named_steps['clf']
    hyperparams = hyperparameters[clf_name]
    rs = RandomizedSearchCV(pipeline, hyperparams, cv=5, scoring='accuracy', n_jobs=-1)
    # rs = GridSearchCV(pipeline, hyperparams, cv=5, scoring='accuracy', n_jobs=-1)
    rs.fit(X_train, y_train)
    
    # Make predictions on train data
    y_train_pred = rs.predict(X_train)
    
    # Compute metrics on train data
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision, train_recall, train_fscore, train_support = precision_recall_fscore_support(y_train, y_train_pred, average='macro')
    
    # Make predictions on test data
    y_test_pred = rs.predict(X_test)
    
    # Compute metrics on test data
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision, test_recall, test_fscore, test_support = precision_recall_fscore_support(y_test, y_test_pred, average='macro')

    # Store results in table
    results_table = results_table.append({
        'Classifier': clf_name,
        'Hyperparameters': rs.best_params_,
        'Train Accuracy': train_accuracy,
        'Train Precision': train_precision,
        'Train Recall': train_recall,
        'Train F1-score': train_fscore,
        'Test Accuracy': test_accuracy,
        'Test Precision': test_precision,
        'Test Recall': test_recall,
        'Test F1-score': test_fscore
    }, ignore_index=True)
    
    # Print progress bar
    remaining_iters = len(pipelines) - (pipelines.index((clf_name, pipeline)) + 1)
    print(f"{remaining_iters} iterations left")
    print("---------------------------------------------------------")

# Keep in mind that by default refit = True
# So, all metrics in the table are for the best found parameters 
  
# Print final results table
print("\nResults table:")
print(results_table)
results_table.to_csv('vh_cat_boost_hp_results2.csv')
files.download('vh_cat_boost_hp_results2.csv')


Classifiers:   0%|          | 0/1 [00:00<?, ?it/s]

Training CatBoost


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0:	learn: 2.4269875	total: 18.3s	remaining: 4m 16s
1:	learn: 2.2215075	total: 37.7s	remaining: 4m 5s
2:	learn: 2.0257787	total: 55.5s	remaining: 3m 41s
3:	learn: 1.8899733	total: 1m 13s	remaining: 3m 21s
4:	learn: 1.7922612	total: 1m 30s	remaining: 3m 1s
5:	learn: 1.7143292	total: 1m 51s	remaining: 2m 47s
6:	learn: 1.6296201	total: 2m 9s	remaining: 2m 28s
7:	learn: 1.5728365	total: 2m 27s	remaining: 2m 9s
8:	learn: 1.5075181	total: 2m 45s	remaining: 1m 50s
9:	learn: 1.4627730	total: 3m 3s	remaining: 1m 31s
10:	learn: 1.4188582	total: 3m 23s	remaining: 1m 13s
11:	learn: 1.3781525	total: 3m 41s	remaining: 55.3s
12:	learn: 1.3428950	total: 3m 59s	remaining: 36.8s
13:	learn: 1.3104799	total: 4m 16s	remaining: 18.3s
14:	learn: 1.2760313	total: 4m 35s	remaining: 0us


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-23-b23f47648c4c>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_table = results_table.append({
Classifiers: 100%|██████████| 1/1 [58:06<00:00, 3486.38s/it]

0 iterations left
---------------------------------------------------------

Results table:
  Classifier                                    Hyperparameters  \
0   CatBoost  {'clf__learning_rate': 0.1, 'clf__iterations':...   

   Train Accuracy  Train Precision  Train Recall  Train F1-score  \
0         0.58491         0.649976      0.509122        0.511107   

   Test Accuracy  Test Precision  Test Recall  Test F1-score  
0       0.518408        0.484362     0.455542       0.436607  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Define pipelines for classifiers
pipelines = [
    ('CatBoost', Pipeline([
        ('preprocessor1', preprocessor1),
        ('clf', cb.CatBoostClassifier(random_state=SEED, 
                                      learning_rate=0.1,
                                      depth=10,
                                      iterations=200))
    ])),

]

In [ ]:
# Without hyp-s tuning
pipelines[0][1].fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0:	learn: 2.4269875	total: 19.5s	remaining: 1h 4m 38s
1:	learn: 2.2215075	total: 47.6s	remaining: 1h 18m 33s
2:	learn: 2.0257787	total: 1m 5s	remaining: 1h 11m 46s
3:	learn: 1.8899733	total: 1m 24s	remaining: 1h 9m 24s
4:	learn: 1.7922612	total: 1m 44s	remaining: 1h 7m 57s
5:	learn: 1.7143292	total: 2m 2s	remaining: 1h 6m 6s
6:	learn: 1.6296201	total: 2m 21s	remaining: 1h 4m 59s
7:	learn: 1.5728365	total: 2m 40s	remaining: 1h 4m 8s
8:	learn: 1.5075181	total: 2m 59s	remaining: 1h 3m 26s
9:	learn: 1.4627730	total: 3m 17s	remaining: 1h 2m 25s
10:	learn: 1.4188582	total: 3m 35s	remaining: 1h 1m 39s
11:	learn: 1.3781525	total: 3m 53s	remaining: 1h 52s
12:	learn: 1.3428950	total: 4m 12s	remaining: 1h 35s
13:	learn: 1.3104799	total: 4m 30s	remaining: 59m 58s
14:	learn: 1.2760313	total: 4m 48s	remaining: 59m 19s
15:	learn: 1.2483500	total: 5m 6s	remaining: 58m 48s
16:	learn: 1.2247784	total: 5m 26s	remaining: 58m 30s
17:	learn: 1.2020403	total: 5m 45s	remaining: 58m 11s
18:	learn: 1.1810595	to

Pipeline(steps=[('preprocessor1',
                 ColumnTransformer(transformers=[('text',
                                                  TfidfVectorizer(tokenizer=<function Vectorization.tokenize_it at 0x7f9b74c13d00>),
                                                  'text'),
                                                 ('num', MinMaxScaler(),
                                                  ['alcohol',
                                                   'wine_name_length',
                                                   'n_grapes']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['category', 'region',
                                                   'is_complex_grape',
                                                   'alcohol_level'])])),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x7f9b518df670>)])

In [ ]:
# Make predictions on train data
y_train_pred_cb = pipelines[0][1].predict(X_train)
# Compute metrics on train data
train_accuracy_cb  = accuracy_score(y_train, y_train_pred_cb)
train_precision_cb , train_recall_cb , train_fscore_cb , train_support_cb  = precision_recall_fscore_support(y_train, y_train_pred_cb, average='macro')

print('CatBoost with the dafault val-s of hyp-s:')
print('Train Accuracy is: ', str(train_accuracy_cb))
print(
    'Train precision, recall, f1, support: ', 
    str(train_precision_cb), " ",
    str(train_recall_cb),  " ",
    str(train_fscore_cb), " ",
    str(train_support_cb))

CatBoost with the dafault val-s of hyp-s:
Train Accuracy is:  0.772410358565737
Train precision, recall, f1, support:  0.8398112814421052   0.7359620298107645   0.767356589051797   None


In [ ]:
# Compute metrics on test data
y_test_pred_cb = pipelines[0][1].predict(X_test)
test_accuracy_cb = accuracy_score(y_test, y_test_pred_cb)
test_precision_cb, test_recall_cb, test_fscore_cb, test_support_cb = precision_recall_fscore_support(y_test, y_test_pred_cb, average='macro')

print('CatBoost with the dafault val-s of hyp-s:')
print('Test Accuracy is: ', str(test_accuracy_cb))
print(
    'Test precision, recall, f1, support: ', 
    str(test_precision_cb), " ",
    str(test_recall_cb),  " ",
    str(test_fscore_cb), " ",
    str(test_support_cb))

CatBoost with the dafault val-s of hyp-s:
Test Accuracy is:  0.5990049751243781
Test precision, recall, f1, support:  0.605725631586453   0.5431779660708314   0.5516690186989464   None


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
